In [55]:
import pandas as pd
import numpy as np
import sklearn.metrics as metricas
from sklearn.model_selection import cross_val_score

## Dataset de CORA

Este dataset contiene dos archivos. El primero es una base de datos de papers que citan a otros papers

In [2]:
citas = pd.read_csv('cora/cora.cites',sep="\t",
    header=None,
    names=["target", "source"])

citas

,target,source
0,35,1033
1,35,103482
2,35,103515
3,35,1050679
4,35,1103960
...,...,...
5424,853116,19621
5425,853116,853155
5426,853118,1140289
5427,853155,853118


El segundo archivo contiene informacién de muchas palabras (términos) asociadas a cada paper, y además una categoría. 

In [3]:
###informacion de los papers de acuerdo a loas palabras que mencionan, junto a su paper id y el tema general

column_names = ["paper_id"] + [f"word_{idx}" for idx in range(1433)] + ["subject"]
papers = pd.read_csv(
    'cora/cora.content', sep="\t", names=column_names,
)
papers

,paper_id,word_0,word_1,word_2,word_3,word_4,word_5,word_6,word_7,word_8,...,word_1424,word_1425,word_1426,word_1427,word_1428,word_1429,word_1430,word_1431,word_1432,subject
0,31336,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,Neural_Networks
1,1061127,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,Rule_Learning
2,1106406,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Reinforcement_Learning
3,13195,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Reinforcement_Learning
4,37879,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Probabilistic_Methods
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2703,1128975,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Genetic_Algorithms
2704,1128977,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Genetic_Algorithms
2705,1128978,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Genetic_Algorithms
2706,117328,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Case_Based


In [4]:
print(papers.subject.value_counts())


subject
Neural_Networks           818
Probabilistic_Methods     426
Genetic_Algorithms        418
Theory                    351
Case_Based                298
Reinforcement_Learning    217
Rule_Learning             180
Name: count, dtype: int64


Vamos a pasar estos subjects a números, y armar los paper_id para que sean consecutivos

In [5]:
class_values = sorted(papers["subject"].unique())
class_idx = {name: id for id, name in enumerate(class_values)}
paper_idx = {name: idx for idx, name in enumerate(sorted(papers["paper_id"].unique()))}

papers["paper_id"] = papers["paper_id"].apply(lambda name: paper_idx[name])
citas["source"] = citas["source"].apply(lambda name: paper_idx[name])
citas["target"] = citas["target"].apply(lambda name: paper_idx[name])
papers["subject"] = papers["subject"].apply(lambda value: class_idx[value])

In [6]:
citas

,target,source
0,0,21
1,0,905
2,0,906
3,0,1909
4,0,1940
...,...,...
5424,1873,328
5425,1873,1876
5426,1874,2586
5427,1876,1874


In [8]:
### solo para guardar los feature names de las cosas que van a ser nuestro dataset para entrenar
feature_names = list(set(papers.columns) - {"paper_id", "subject"})


Nuestra tarea consiste en predecir la línea "subject" (el tema del paper), basándonos por ahora solo en el vector de ocurrencias de palabras. Con esto, dividimos el set de training y test en una matriz x y un vector y con los subjects. 

### Un primer clasificador

Vamos a intentar predecir el subject de cada paper, usando solo la información en el dataframe de papers (no las citas). Vamos a usar RandomForest, y como métrica el score F1. Como esta clasificación es multiclase, tomamos el promedio de los scores F1 de cada clase (esto corresponde al scoring f1_macro).

In [9]:
from sklearn.ensemble import RandomForestClassifier


In [61]:
score_rf = cross_val_score(RandomForestClassifier(n_estimators=100, random_state=13), papers[feature_names], papers["subject"], scoring='f1_macro')
score_rf

array([0.73340208, 0.74247913, 0.70065171, 0.7166704 , 0.72301317])

### Agregando información de citas

Lo primero es usar networkx, una librería de grafos, para almacenar nuestro grafo en memoria. Luego vamos a llamar a Node2Vec para conseguir embeddings. 

In [12]:
import networkx as nx

In [33]:
#### armamos un grafo dirigido con la estructura de citas
G = nx.from_pandas_edgelist(citas, source="source", target="target",create_using=nx.DiGraph())


In [22]:
#### vamos a condensar la información de las palabras del dataframe paper en una sola dimensión, 
### la que usaremos como el peso de cada nodo para Node2Vec. 
from sklearn.decomposition import PCA

In [24]:
pca = PCA(n_components=1)

Features_1D = pca.fit_transform(papers[feature_names])

In [27]:
### y agregamos esa info al dataframe de papers. 
papers["weight"] = Features_1D

In [34]:
### Ahora agregamos la información de cada nodo a nuestro grafo. Solo agregamos el peso 

subjects_dict = dict(zip(papers['paper_id'], papers['weight']))
nx.set_node_attributes(G, subjects_dict, 'weight')


Podemos imprimir valores nodo a nodo usando networkx

In [35]:
for node_id in G.nodes:
    node_data = G.nodes[node_id]
    print(f"Node {node_id} data: {node_data}")

    neighbors = list(G.neighbors(node_id))
    print(f"Node {node_id} neighbors: {neighbors}")
    break

Node 21 data: {'weight': 0.7571532981634186}
Node 21 neighbors: [0, 557, 582]


Ahora, ya con nuestro grafo, podemos llamar a Node2Vec

In [36]:
from node2vec import Node2Vec

### dimensions es el tamaño de los features. Walk_length el tamaño de las caminatas, y 
### num_walks la cantidad de caminatas. 
node2vec = Node2Vec(G, dimensions=16, walk_length=10, num_walks=20)

Computing transition probabilities:   0%|          | 0/2708 [00:00<?, ?it/s]

Generating walks (CPU: 1): 100%|████████████████| 20/20 [00:00<00:00, 68.50it/s]


In [37]:
### Con fit calculamos embeddings. Min_count y batch_words son parametros heredados de
### Word2vec, pero instanciados aca de forma mas pequeña. 

model = node2vec.fit(window=10, min_count=1, batch_words=4)


In [38]:
embeddings = {str(node): model.wv[str(node)] for node in G.nodes()}

In [54]:
### Armamos un dataframe con los embeddings y lo concatenamos al de papers

columns_embedding =['paper_id'] + ['dim_'+str(i) for i in range(0,16)]
df = pd.DataFrame(
[(int(node), *values) for node, values in embeddings.items()], columns=columns_embedding)

papers_embeddings = pd.merge(df,papers,on='paper_id')



      paper_id     dim_0     dim_1     dim_2     dim_3     dim_4     dim_5  \
0           21 -0.694267 -0.357350  1.640686  0.749505  0.525538  0.355988   
1            0  0.073137  0.523458  1.899398  2.115041  0.099030 -0.209619   
2          905 -0.209860  0.036354  1.168403  0.732985  0.181481  0.004719   
3          906 -0.036907 -0.418786  1.743310  0.917336 -0.022319  0.291986   
4         1909 -0.646281 -0.202158  0.631465  0.148104  0.046897 -0.050843   
...        ...       ...       ...       ...       ...       ...       ...   
2703      2585  0.001337  0.006236  0.703431 -0.338559  0.226602  0.469466   
2704      1871 -1.794383 -0.476002  0.450609  2.387101  1.451134  1.694079   
2705      1876 -1.137313 -0.794808  0.136642  1.496496  0.818069  1.295134   
2706      1872 -1.622827 -0.370402  0.730027  2.495121  1.031816  1.666858   
2707      1874 -0.985864 -0.588732  0.127847  1.381832  0.630383  1.058639   

         dim_6     dim_7     dim_8     dim_9    dim_10    dim_1

In [63]:
### y ahora vemos que tal el clasificador cuando agregamos información de 
### las citas

datos_utiles = feature_names + ['dim_'+str(i) for i in range(0,16)]

score_rf = cross_val_score(RandomForestClassifier(n_estimators=100, random_state=13), papers_embeddings[datos_utiles], papers_embeddings["subject"], scoring='f1_macro')
score_rf

array([0.77466769, 0.7503533 , 0.7786987 , 0.7294128 , 0.717947  ])